In [1]:
import re
from difflib import get_close_matches

from conda.instructions import ACTION_CODES
from tabulate import tabulate

In [9]:
LOCATION_PATTERN = r"[a-zA-Z]{3}(?:-[a-zA-Z]{2,3})?"
LOCATION_PATTERN = r"[a-zA-Z]{3}(?:-[nswe]{1}c)?"
LOCATION_PATTERN = r"^[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?$"


In [10]:
# test location pattern

test = [
	"mun", "bul", "bul-sc", "stp-nc", "stp-sc",
	"bul-ec", "bul-nc", "bul-sc", "bul-ec", "stp-nc",
	"stp-sc", "bud", "par", "lon", "edi", "lvp",
	"ber", "mun", "mar", "spa-nc", "spa-sc",]
fail_tests = ['mun-ac', 'asfd', 'abs-x', 'abs-xx']
for t in test:
	if re.match(LOCATION_PATTERN, t):
		print(f"'{t}' matches the location pattern.")
	else:
		print(f"'{t}' does NOT match the location pattern.")

for t in fail_tests:
	if re.match(LOCATION_PATTERN, t):
		print(f"'{t}' matches the location pattern.")
	else:
		print(f"'{t}' does NOT match the location pattern.")


'mun' matches the location pattern.
'bul' matches the location pattern.
'bul-sc' matches the location pattern.
'stp-nc' matches the location pattern.
'stp-sc' matches the location pattern.
'bul-ec' matches the location pattern.
'bul-nc' matches the location pattern.
'bul-sc' matches the location pattern.
'bul-ec' matches the location pattern.
'stp-nc' matches the location pattern.
'stp-sc' matches the location pattern.
'bud' matches the location pattern.
'par' matches the location pattern.
'lon' matches the location pattern.
'edi' matches the location pattern.
'lvp' matches the location pattern.
'ber' matches the location pattern.
'mun' matches the location pattern.
'mar' matches the location pattern.
'spa-nc' matches the location pattern.
'spa-sc' matches the location pattern.
'mun-ac' does NOT match the location pattern.
'asfd' does NOT match the location pattern.
'abs-x' does NOT match the location pattern.
'abs-xx' does NOT match the location pattern.


In [21]:

LOCATION_PATTERN = r"^[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?$"
code = rf"^(?P<start>{LOCATION_PATTERN})\s+convoys?\s+(?P<mid>{LOCATION_PATTERN})\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$"

tests = [
	"mun convoys bul to con",
	"mun convoy bul to con",
]

print(tabulate([[x, str(re.compile(code, re.IGNORECASE).match(x))] for x in tests]))

----------------------  ----
mun convoys bul to con  None
mun convoy bul to con   None
----------------------  ----


In [34]:
import re
from tabulate import tabulate

UNIT_PATTERN = r"P<unit>(?:[A|F|a|f|army|fleet|troop])"

LOCATION_PATTERN = r"[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?"

IDENT_PATTERN = rf"({UNIT_PATTERN}\s)?P<loc>{LOCATION_PATTERN})"

# Synonyms for the "convoy" part
CONVOY_SYNONYMS = r"(?:convoy|convoys|convey|c|con|conv|convoying)"
SUPPORT_SYNONYMS = r"(?:support|supports|sup|s|spt|sp|supprt|supp|sptg|sptng)"
TO_SYNONYMS = r"(?:to|->|t|-|>)"

# Build the overall pattern
code = rf"^(?{IDENT_PATTERN})\s+{CONVOY_SYNONYMS}\s+(?P<mid>{LOCATION_PATTERN})\s+{TO_SYNONYMS}\s+(?P<end>{LOCATION_PATTERN})$"

tests = [
	"mun convoys bul to con",
	"mun convoy bul to con",
	"f kie convoy bul to con",
	"mun convey bul-ec to con",
	"mun convoy bul - con",
	"mun c bul - con",
	"mun c bul t con",
	"mun c bul > con",
	"mun c bul-ec t con",
]

results = []
for txt in tests:
	pattern = re.compile(code, re.IGNORECASE)
	match = pattern.match(txt)
	match_found = bool(match)
	match_groups = match.groupdict() if match else None
	results.append([txt, match_found, match_groups])

print(tabulate(results, headers=["Order", "Match?"]))


error: bad character in group name 'P<unit>(?:[A|F|a|f|army|fleet|troop]' at position 4

In [59]:
import re
from tabulate import tabulate

UNIT_PATTERN = r"(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy)"

LOCATION_PATTERN = r"[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?"

IDENT_PATTERN = rf"(?:(?P<unit>{UNIT_PATTERN})\s+)?(?P<loc>{LOCATION_PATTERN})"
TARGET_PATTERN = rf"(?:(?P<tunit>{UNIT_PATTERN})\s+)?(?P<target>{LOCATION_PATTERN})"

CONVOY_SYNONYMS = r"(?:convoy|convoys|convey|c|con|conv|convoying)"
# SUPPORT_SYNONYMS = r"(?:support|supports|sup|s|spt|sp|supprt|supp|sptg|sptng|:)"
SUPPORT_SYNONYMS = r"(?:support|supports|sup|s|spt|sp|supprt|supp|sptg|sptng)"
SUPPORT_HOLD_SYNONYMS = rf"((?:sh|supporthold|support-hold|s-h|s-hold|shold|s-hld|shld)|{SUPPORT_SYNONYMS})"
TO_OPTIONS = 'to|->|t|-|>'
MOVE_OPTIONS = r"move|moves|m|mv|mov|moving"
TO_SYNONYMS = rf"(?:{TO_OPTIONS})"
MOVE_SYNONYMS = rf"(?:{MOVE_OPTIONS})"
MOVETO_SYNONYMS = rf"({MOVE_SYNONYMS}\s+)?{TO_SYNONYMS}"
HOLD_SYNONYMS = rf"(?:hold|holds|h|hd|hld|holding|h)"
TOHOLD_SYNONYMS = rf"\s+({TO_SYNONYMS}\s+{HOLD_SYNONYMS})"

DISBAND_SYNONYMS = rf"(?:disband|disbands|d|db|dnd|disb|disbanding|dndg)"
RETREAT_SYNONYMS = rf"(?:retreat|retreats|r|rt|rtr|ret|retreating|rtrg)"
RETREATTO_SYNONYMS = rf"({RETREAT_SYNONYMS}\s+)?{TO_SYNONYMS}"
BUILD_SYNONYMS = rf"(?:build|builds|b|bd|bld|building|bldg)"

HOLD_PATTERNS = rf"({IDENT_PATTERN}(\s+{HOLD_SYNONYMS})?)|({HOLD_SYNONYMS}\s+{IDENT_PATTERN})"
MOVE_PATTERN = rf"({MOVE_SYNONYMS}\s+)?{IDENT_PATTERN}(\s+{MOVETO_SYNONYMS})?\s+{TARGET_PATTERN}"

CONVOY_PATTERN = rf"({IDENT_PATTERN}\s+{CONVOY_SYNONYMS}\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})\s+{LOCATION_PATTERN})|({CONVOY_SYNONYMS}\s+{IDENT_PATTERN}:?\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})?\s+{LOCATION_PATTERN})"
SUPPORTS_PATTERN = rf"({IDENT_PATTERN}(?:(\s+{SUPPORT_SYNONYMS})|:)\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})\s+{LOCATION_PATTERN})|({SUPPORT_SYNONYMS}\s+{IDENT_PATTERN}:?\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})?\s+{LOCATION_PATTERN})"

SUPPORT_HOLDS_PATTERNS = rf"({IDENT_PATTERN}((\s+{SUPPORT_HOLD_SYNONYMS})|:)\s+{HOLD_SYNONYMS}\s+{LOCATION_PATTERN})|({SUPPORT_HOLD_SYNONYMS}\s+{IDENT_PATTERN}(\s+{HOLD_SYNONYMS})?\s+{LOCATION_PATTERN})"

BUILD_PATTERNS = rf"({IDENT_PATTERN}(\s+{BUILD_SYNONYMS})?)|({BUILD_SYNONYMS}\s+{IDENT_PATTERN})"
DISBAND_PATTERN = rf"({IDENT_PATTERN}(\s+{DISBAND_SYNONYMS})?)|({DISBAND_SYNONYMS}\s+{IDENT_PATTERN})"
RETREAT_PATTERN = rf"({RETREAT_SYNONYMS}\s+)?{IDENT_PATTERN}(\s+{RETREATTO_SYNONYMS})?\s+{LOCATION_PATTERN}"

In [39]:
def regex_test(code, tests):
	results = []
	pattern = re.compile(code, re.IGNORECASE)
	for txt in tests:
		match = pattern.match(txt)
		match_found = bool(match)
		match_groups = match.groupdict() if match else None
		results.append([txt, match_found, match_groups])
	print(tabulate(results, headers=["Order", "Match?", "Groups"]))

In [43]:
move_tests = [
	"mun to con",
	"mun moves to con",
	"army mun moves to con",
	"move a mun - con",
	"m mun t con",
	"f mun - con",
]
regex_test(MOVE_PATTERN, move_tests)

Order             Match?    Groups
----------------  --------  ------------------------------------------------------------
mun to con        True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'con'}
move a mun - con  True      {'unit': 'a', 'loc': 'mun', 'tunit': None, 'target': 'con'}
m mun t con       True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'con'}
f mun - con       True      {'unit': 'f', 'loc': 'mun', 'tunit': None, 'target': 'con'}


In [56]:
support_tests = [
	"mun supports bul to con",
	"mun support bul to con",
	"f kie support bul to con",
	"mun s bul-ec to con",
	"mun support bul - con",
	"mun : bul - con",
	"mun: bul - con",
]
regex_test(SUPPORTS_PATTERN, support_tests)

Order                     Match?    Groups
------------------------  --------  ---------------------------------------------------------------
mun supports bul to con   True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul'}
mun support bul to con    True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul'}
f kie support bul to con  True      {'unit': 'f', 'loc': 'kie', 'tunit': None, 'target': 'bul'}
mun s bul-ec to con       True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul-ec'}
mun support bul - con     True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul'}
mun : bul - con           True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul'}
mun: bul - con            True      {'unit': None, 'loc': 'mun', 'tunit': None, 'target': 'bul'}


In [ ]:
convoy_tests = [
	"mun convoys bul to con",
	"mun convoy bul to con",
	"f kie convoy bul to con",
	"mun convey bul-ec to con",
	"mun convoy bul - con",
	"mun c bul - con",
	"mun c bul t con",
	"mun c bul > con",
	"mun c bul-ec t con",
]
regex_test(CONVOY_SYNONYMS, move_tests)

In [37]:

# Finally, build the overall pattern for a convoy command:
#   ^ : start of string
#   IDENT_PATTERN : optional unit + location
#   \s+ : space
#   CONVOY_SYNONYMS : "convoy", "convoys", "convey", etc.
#   \s+ : space
#   (?P<mid>LOCATION_PATTERN) : mid location
#   \s+ : space
#   TO_SYNONYMS : "to", "->", "t", "-", ">"
#   \s+ : space
#   (?P<end>LOCATION_PATTERN) : end location
#   $ : end of string
code = (
	rf"^{IDENT_PATTERN}\s+{CONVOY_SYNONYMS}\s+"
	rf"(?P<mid>{LOCATION_PATTERN})\s+"
	rf"{TO_SYNONYMS}\s+"
	rf"(?P<end>{LOCATION_PATTERN})$"
)

move_tests = [
	"mun to con",
	"move mun - con",
	"m mun t con",
]

# Test strings that should all match
tests = [
	"mun convoys bul to con",
	"mun convoy bul to con",
	"f kie convoy bul to con",
	"mun convey bul-ec to con",
	"mun convoy bul - con",
	"mun c bul - con",
	"mun c bul t con",
	"mun c bul > con",
	"mun c bul-ec t con",
]

results = []
pattern = re.compile(code, re.IGNORECASE)

for txt in tests:
	match = pattern.match(txt)
	match_found = bool(match)
	match_groups = match.groupdict() if match else None
	results.append([txt, match_found, match_groups])

print(tabulate(results, headers=["Order", "Match?", "Groups"]))


Order                     Match?    Groups
------------------------  --------  -----------------------------------------------------------
mun convoys bul to con    True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
mun convoy bul to con     True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
f kie convoy bul to con   True      {'unit': 'f', 'loc': 'kie', 'mid': 'bul', 'end': 'con'}
mun convey bul-ec to con  True      {'unit': None, 'loc': 'mun', 'mid': 'bul-ec', 'end': 'con'}
mun convoy bul - con      True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
mun c bul - con           True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
mun c bul t con           True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
mun c bul > con           True      {'unit': None, 'loc': 'mun', 'mid': 'bul', 'end': 'con'}
mun c bul-ec t con        True      {'unit': None, 'loc': 'mun', 'mid': 'bul-ec', 'end': 'con'}


In [ ]:

LOCATION_PATTERN = r"^[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?$"
# 2) -------------------------------------------------------
# Define the accepted patterns for each order type
# (You can add as many variations as you like per order type.)

ORDER_PATTERNS = {
	"move": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$",
	],
	"support_move": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+supports\s+(?P<mid>{LOCATION_PATTERN})\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$",
	],
	"support_hold": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+supports?\s+hold(s?)\s+(?P<mid>{LOCATION_PATTERN})$",
		# Variation: "[X] supports [Y] holds", or "[X] support holds [Y]" (just to illustrate)
		rf"^(?P<start>{LOCATION_PATTERN})\s+supports?\s+holds?\s+(?P<mid>{LOCATION_PATTERN})$",
	],
	"convoy": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+convoys?\s+(?P<mid>{LOCATION_PATTERN})\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$",
		rf"^(?P<start>{LOCATION_PATTERN})\s+convoy?\s+(?P<mid>{LOCATION_PATTERN})\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$",
	],
	"hold": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+holds?$",
	],
	"retreat": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+retreats?\s+(?:to|->)\s+(?P<end>{LOCATION_PATTERN})$",
	],
	"disband": [
		rf"^(?P<start>{LOCATION_PATTERN})\s+disbands?$",
	],
	"build": [
		rf"^build\s+(?P<unit>[AF])\s+in\s+(?P<start>{LOCATION_PATTERN})$",
		# "Build A in LVP", "Build F in MAR", ...
		rf"^build\s+(?P<unit>[a-zA-Z]+)\s+in\s+(?P<start>{LOCATION_PATTERN})$"
	],
}

# Make them case-insensitive, so we can do re.IGNORECASE
COMPILED_PATTERNS = {
	order_type: [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
	for order_type, patterns in ORDER_PATTERNS.items()
}

# 3) -------------------------------------------------------
# Optionally, define known location codes & known keywords to assist with fuzzy matching
KNOWN_LOCATIONS = [
	"bul-sc", "bul-ec", "stp-nc", "stp-sc", "bud", "par", "lon", "edi", "lvp",
	"ber", "mun", "mar", "spa-nc", "spa-sc",
	# etc...
]
KNOWN_KEYWORDS = [
	"to", "supports", "support", "hold", "holds", "convoy", "convoys",
	"retreat", "retreats", "disband", "disbands", "build", "in"
]
# Also consider short tokens like "->" etc.


def suggest_fuzzy_matches(token: str, vocabulary, cutoff=0.75, max_suggestions=3):
	"""
	Return a list of near matches from the given 'vocabulary' for 'token'.
	cutoff: minimum closeness ratio
	"""
	return get_close_matches(token, vocabulary, n=max_suggestions, cutoff=cutoff)


# 4) -------------------------------------------------------
def parse_order(order_text: str):
	"""
	Attempt to match the order_text against all known order patterns.
	If matched, return a dictionary describing the order:
		{
			"type": ...,
			"groups": { "start": ..., "end": ..., "mid": ..., ...}
		}
	If not matched, raise a ValueError explaining why and (optionally)
	suggest possible corrections.
	"""
	original_text = order_text.strip()

	# Try each order type
	for order_type, pattern_list in COMPILED_PATTERNS.items():
		for pattern in pattern_list:
			match = pattern.match(original_text)
			if match:
				# Extract matched groups
				return {
					"type": order_type,
					"groups": match.groupdict()
				}

	# 5) ----------------------------------------------------
	# If we get here, parsing failed. Let’s produce a thorough error message.
	# We can do the following:
	#   1. Tokenize the input
	#   2. Fuzzy search each token among known locations or known keywords
	#   3. Build suggestions
	tokens = original_text.split()

	# If tokens is empty or weird, we can short-circuit:
	if not tokens:
		raise ValueError("Parsing error: No text provided.")

	suggestions = []
	# For each token, see if we can find a close match in locations or keywords:
	for tok in tokens:
		loc_suggestions = suggest_fuzzy_matches(tok.lower(), KNOWN_LOCATIONS)
		key_suggestions = suggest_fuzzy_matches(tok.lower(), KNOWN_KEYWORDS)

		tok_suggestion = None
		if loc_suggestions:
			tok_suggestion = f"Location? Did you mean: {', '.join(loc_suggestions)}"
		elif key_suggestions:
			tok_suggestion = f"Keyword? Did you mean: {', '.join(key_suggestions)}"
		if tok_suggestion:
			suggestions.append(f"'{tok}' → {tok_suggestion}")

	# Build an explanatory message with suggestions:
	# E.g. "Could not parse your order: [text]. Possibly invalid keywords or structure."
	error_message = []
	error_message.append(f"Parsing error: Could not parse the order '{original_text}'.")
	error_message.append("Check your order format. Examples of valid formats include:")
	error_message.append(" • 'BER to MUN'")
	error_message.append(" • 'ber supports mun to bur'")
	error_message.append(" • 'bul-sc convoys mar to spa-sc'")
	error_message.append(" • 'build A in LVP'")
	error_message.append(" • 'lon holds'")

	# If we found suggestions for any tokens:
	if suggestions:
		error_message.append("\nPossible corrections based on fuzzy matching:")
		for s in suggestions:
			error_message.append("  - " + s)

	raise ValueError("\n".join(error_message))


# 6) -------------------------------------------------------
# Example usage of parse_order

def main():
	tests = [
		"MUN to BER",
		"par -> mar",
		"lon support edi to lvp",
		"Lon convoys Mar to Spa-sc",
		"xyz to abc",  # clearly invalid
		"build a in lvp",
		"BUL-SC convoys mar to spa-sc",
		"ber holds",
		"some nonsense text"
	]

	for t in tests:
		print("------------------------------------")
		print(f"Order: {t}")
		try:
			result = parse_order(t)
			print("SUCCESS:", result)
		except ValueError as e:
			print("ERROR:", e)
	print("------------------------------------")

if __name__ == "__main__":
	main()


In [63]:
import re
from tabulate import tabulate

# ------------------------------------------------------------------------------------
# 1. UNIT & LOCATION SYNONYMS
# ------------------------------------------------------------------------------------
UNIT_PATTERN = r"(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy)"

# Original location pattern: "mun", "spa-nc", etc.
LOCATION_PATTERN = r"[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?"

# Many players use parentheses or brackets around location, e.g. "(con)" or "[ank]"
# We'll allow *one* optional pair of parentheses or brackets around the location.
# e.g. "ank", "(ank)", "[ank]"
LOCATION_WRAPPED = rf"[\(\[]?\s*{LOCATION_PATTERN}\s*[\)\]]?"

# Some players say "army in mun", "fleet at bre". So we allow (in|at) between unit + loc.
# Combine them into one capture group:  optional UNIT ( plus optional "in|at" ) + location
# Named groups remain the same: "unit" and "loc".
IDENT_PATTERN = rf"(?:(?P<unit>{UNIT_PATTERN})(?:\s+(?:in|at))?\s+)?(?P<loc>{LOCATION_WRAPPED})"

# For a second location (in move/support commands), do the same approach:
TARGET_PATTERN = rf"(?:(?P<tunit>{UNIT_PATTERN})(?:\s+(?:in|at))?\s+)?(?P<target>{LOCATION_WRAPPED})"

# ------------------------------------------------------------------------------------
# 2. GENERAL SYNONYMS & OPTIONAL PHRASES
# ------------------------------------------------------------------------------------
# Expand 'TO_OPTIONS' to allow "=>", "attack", "att", "for"
# (so e.g. "army wal attack lvp", "lon s h for nwy", "wal => lvp")
TO_OPTIONS = r"to|->|t|-|>|=>|attack|att|for"

# We can also allow "from" before the first location in a move:
FROM_OPT = r"(?:from\s+)?"

# And we can allow "via" an extra location at the end of some orders (Convoy, Retreat):
# e.g., "convoys bul to ank via bla"
# We'll define a group that optionally matches "via" + a location.
VIA_OPT = rf"(?:\s+via\s+{LOCATION_WRAPPED})?"

# Move synonyms remain basically the same.
MOVE_OPTIONS = r"move|moves|m|mv|mov|moving"
MOVE_SYNONYMS = rf"(?:{MOVE_OPTIONS})"

# 'to' synonyms can incorporate "to|->|...|=>"
TO_SYNONYMS = rf"(?:{TO_OPTIONS})"

# For "move to" combos, e.g. "army mun move to par" or "army mun m -> par"
MOVETO_SYNONYMS = rf"({MOVE_SYNONYMS}\s+)?{TO_SYNONYMS}"

# 'hold' synonyms unchanged, but we might allow parentheses, e.g., "h", "holds", ...
HOLD_SYNONYMS = r"(?:hold|holds|h|hd|hld|holding|h)"

# We can keep the rest of synonyms but expand them as you see fit
CONVOY_SYNONYMS = r"(?:convoy|convoys|convey|c|con|conv|convoying)"
SUPPORT_SYNONYMS = r"(?:support|supports|sup|s|spt|sp|supprt|supp|sptg|sptng)"
SUPPORT_HOLD_SYNONYMS = rf"((?:sh|supporthold|support-hold|s-h|s-hold|shold|s-hld|shld)|{SUPPORT_SYNONYMS})"

DISBAND_SYNONYMS = r"(?:disband|disbands|d|db|dnd|disb|disbanding|dndg)"
RETREAT_SYNONYMS = r"(?:retreat|retreats|r|rt|rtr|ret|retreating|rtrg)"
RETREATTO_SYNONYMS = rf"({RETREAT_SYNONYMS}\s+)?{TO_SYNONYMS}"

BUILD_SYNONYMS = r"(?:build|builds|b|bd|bld|building|bldg)"

# ------------------------------------------------------------------------------------
# 3. REVISED ORDER-TYPE PATTERNS
# ------------------------------------------------------------------------------------
# 3.1 HOLD
#
# Allowed forms:
#  - "[unit] [location] hold"
#  - "hold [unit] [location]"
#  - with optional "in|at", parentheses around location, etc.
#
HOLD_PATTERNS = rf"({IDENT_PATTERN}(\s+{HOLD_SYNONYMS})?)|({HOLD_SYNONYMS}\s+{IDENT_PATTERN})"

# 3.2 MOVE
#
# We'll allow an optional "from" before the origin, plus all synonyms for "move" / "to".
# E.g. "army from mun -> par", "army mun move to par", "f stp => nwg", etc.
#
MOVE_PATTERN = rf"({MOVE_SYNONYMS}\s+)?{FROM_OPT}{IDENT_PATTERN}(\s+{MOVETO_SYNONYMS})?\s+{TARGET_PATTERN}"

# 3.3 CONVOY
#
# We'll allow an optional "via" location at the end (VIA_OPT).
# For example: "con convoys bul to ank via bla"
#
CONVOY_PATTERN = (
	rf"({IDENT_PATTERN}\s+{CONVOY_SYNONYMS}\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})\s+{LOCATION_WRAPPED}{VIA_OPT})"
	rf"|"
	rf"({CONVOY_SYNONYMS}\s+{IDENT_PATTERN}:?\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})?\s+{LOCATION_WRAPPED}{VIA_OPT})"
)

# 3.4 SUPPORT (MOVE)
#
# We might allow 'attack' or 'att' in place of 'to'. Already handled by TO_OPTIONS.
#   e.g. "F lon support A wal attack lvp"
# Everything else remains the same, but now parentheses & bracket usage is recognized.
#
SUPPORTS_PATTERN = (
	rf"({IDENT_PATTERN}(?:(\s+{SUPPORT_SYNONYMS})|:)\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})\s+{LOCATION_WRAPPED})"
	rf"|"
	rf"({SUPPORT_SYNONYMS}\s+{IDENT_PATTERN}:?\s+{TARGET_PATTERN}(\s+{TO_SYNONYMS})?\s+{LOCATION_WRAPPED})"
)

# 3.5 SUPPORT (HOLD)
#
# We'll allow "for" as a synonym of "to" if you want that specifically for hold:
# but we already put "for" in TO_OPTIONS, so "s hold for nwy" might parse fine.
#
SUPPORT_HOLDS_PATTERNS = (
	rf"({IDENT_PATTERN}((\s+{SUPPORT_HOLD_SYNONYMS})|:)\s+{HOLD_SYNONYMS}\s+{LOCATION_WRAPPED})"
	rf"|"
	rf"({SUPPORT_HOLD_SYNONYMS}\s+{IDENT_PATTERN}(\s+{HOLD_SYNONYMS})?\s+{LOCATION_WRAPPED})"
)

# 3.6 BUILD
#
# Some players do "build f in mun". Now we handle that because IDENT_PATTERN itself
# allows "f in mun".
#
BUILD_PATTERNS = rf"({IDENT_PATTERN}(\s+{BUILD_SYNONYMS})?)|({BUILD_SYNONYMS}\s+{IDENT_PATTERN})"

# 3.7 DISBAND
#
# Similarly, "disband fleet in par" should now work because of IDENT_PATTERN's "in|at".
#
DISBAND_PATTERN = rf"({IDENT_PATTERN}(\s+{DISBAND_SYNONYMS})?)|({DISBAND_SYNONYMS}\s+{IDENT_PATTERN})"

# 3.8 RETREAT
#
# Optionally allow "via" a location (like "army mun retreat via tyr to boh"),
# and "=>", "attack", or "for" synonyms for "to".
#
RETREAT_PATTERN = rf"({RETREAT_SYNONYMS}\s+)?{IDENT_PATTERN}(\s+{RETREATTO_SYNONYMS})?\s+{LOCATION_WRAPPED}{VIA_OPT}"

# ------------------------------------------------------------------------------------
# 4. DEMO TESTS
# ------------------------------------------------------------------------------------

# Minimal examples showing that newly allowed structures now match.
patterns_to_test = {
	"Hold": HOLD_PATTERNS,
	"Move": MOVE_PATTERN,
	"Convoy": CONVOY_PATTERN,
	"Support (move)": SUPPORTS_PATTERN,
	"Support (hold)": SUPPORT_HOLDS_PATTERNS,
	"Build": BUILD_PATTERNS,
	"Disband": DISBAND_PATTERN,
	"Retreat": RETREAT_PATTERN,
}

sample_inputs = [
	# Examples that used to fail but now pass:
	("Hold",   "army in mun hold"),
	("Hold",   "A [mun] holds"),
	("Move",   "army from mun to par"),
	("Move",   "army mun => par"),
	("Convoy", "F (con) c A [bul] => (ank) via bla"),
	("Convoy", "con convoys bul to ank via bla"),
	("Support (move)", "F lon supports A wal attack lvp"),
	("Support (move)", "lon s wal => lvp"),
	("Support (hold)", "lon s h for nwy"),
	("Build",  "build f in mun"),
	("Build",  "army at lvp b"),
	("Disband","disband fleet in par"),
	("Retreat","army mun retreat via tyr => boh"),
	("Retreat","army mun r => boh"),
]

for order_type, text in sample_inputs:
	pattern = patterns_to_test[order_type]
	try:
		match = re.compile(pattern, re.IGNORECASE).match(text)
	except:
		print(order_type, pattern)
		raise
	print(f"{order_type:<14} | {text:<40} | MATCH={bool(match)}")


Hold (?:((?:(?P<unit>(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy))(?:\s+(?:in|at))?\s+)?(?P<loc>[\(\[]?\s*[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?\s*[\)\]]?)(\s+(?:hold|holds|h|hd|hld|holding|h))?)|((?:hold|holds|h|hd|hld|holding|h)\s+(?:(?P<unit>(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy))(?:\s+(?:in|at))?\s+)?(?P<loc>[\(\[]?\s*[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?\s*[\)\]]?)))


error: redefinition of group name 'unit' as group 6; was group 2 at position 230

In [64]:
import re

UNIT_PATTERN = r"(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy)"
LOCATION_PATTERN = r"[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?"

# Possibly allow parentheses/brackets around location:
LOCATION_WRAPPED = rf"[\(\[]?\s*{LOCATION_PATTERN}\s*[\)\]]?"

# We might allow "in|at" between the unit and the location
# e.g. "army in mun hold"
HOLD_SYNONYMS = r"(?:hold|holds|h|hd|hld|holding|h)"

# Pattern A: "unit location [hold]"
#   - Example: "F mun holds", "army in bur h"
HOLD_PATTERN_A = rf"""
^
(?P<unit>{UNIT_PATTERN})
(?:\s+(?:in|at))?
\s+
(?P<loc>{LOCATION_WRAPPED})
(?:\s+(?P<hold_verb>{HOLD_SYNONYMS}))?
$
"""

# Pattern B: "[hold] unit location"
#   - Example: "hold f mun", "holds army (par)"
HOLD_PATTERN_B = rf"""
^
(?P<hold_verb>{HOLD_SYNONYMS})
\s+
(?P<unit>{UNIT_PATTERN})
(?:\s+(?:in|at))?
\s+
(?P<loc>{LOCATION_WRAPPED})
$
"""

# We'll compile these patterns (verbose & case-insensitive):
HOLD_PATTERNS = [
	re.compile(HOLD_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(HOLD_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]


def parse_hold_order(order: str):
	"""
	Tries each HOLD pattern in sequence. If any match,
	returns a dict with the relevant named groups.
	Otherwise, returns None or raises an error.
	"""
	text = order.strip()
	for pat in HOLD_PATTERNS:
		m = pat.match(text)
		if m:
			return m.groupdict()
	return None


if __name__ == "__main__":
	test_orders = [
		"F mun holds",           # A
		"mun hold",             # A, but no 'unit' => might fail (no unit though)
		"army bur h",           # A
		"bur holds",            # A, missing 'army' => might fail unless you want location to be 'bur' and no unit
		"hold f mun",           # B
		"holds army (par)",     # B
		"h lvp",                # minimal
		"Na stp hold",          # A
		"army in mun hold",     # A
	]

	for order_text in test_orders:
		result = parse_hold_order(order_text)
		print(f"'{order_text}' => MATCH={bool(result)}; groups={result}")


'F mun holds' => MATCH=True; groups={'unit': 'F', 'loc': 'mun', 'hold_verb': 'holds'}
'mun hold' => MATCH=False; groups=None
'army bur h' => MATCH=True; groups={'unit': 'army', 'loc': 'bur', 'hold_verb': 'h'}
'bur holds' => MATCH=False; groups=None
'hold f mun' => MATCH=True; groups={'hold_verb': 'hold', 'unit': 'f', 'loc': 'mun'}
'holds army (par)' => MATCH=True; groups={'hold_verb': 'holds', 'unit': 'army', 'loc': '(par)'}
'h lvp' => MATCH=False; groups=None
'Na stp hold' => MATCH=True; groups={'unit': 'Na', 'loc': 'stp', 'hold_verb': 'hold'}
'army in mun hold' => MATCH=True; groups={'unit': 'army', 'loc': 'mun', 'hold_verb': 'hold'}


In [110]:
import re

# ----------------------------------------------------------------------------------
# 1) HELPER PATTERNS & SYNONYMS
# ----------------------------------------------------------------------------------

# Simple synonyms and patterns for units, locations, etc.
UNIT_PATTERN = r"(?:a|t|f|n|ar|tr|fl|na|army|troop|fleet|navy)"
LOCATION_PATTERN = r"[A-Za-z]{3}(?:-(?:nc|sc|ec|wc))?"

HOLD_SYNONYMS = r"(?:hold|holds|h|hd|hld|holding)"
MOVE_SYNONYMS = r"(?:move|moves|m|mv|mov|moving)"
TO_SYNONYMS    = r"(?:to|->|-|>)"
SUPPORT_SYNONYMS = r"(?:support|supports|sup|s|spt|sp|supp)"
CONVOY_SYNONYMS  = r"(?:convoy|convoys|convey|c|conv|convoying)"
BUILD_SYNONYMS   = r"(?:build|builds|b|bd|bld|building|bldg)"
DISBAND_SYNONYMS = r"(?:disband|disbands|d|db|disb|disbanding)"
RETREAT_SYNONYMS = r"(?:retreat|retreats|r|rt|rtr|ret|retreating)"

# A small convenience: optionally capture "in|at" between unit & location
# e.g., "army in mun" or "fleet at lon"
IN_AT_OPT = r"(?:\s+(?:in|at))?"

A_AN_OPT = r"(?:\s+(?:a|an))?"

# If you want parentheses or brackets around location, you might do:
#   LOCATION_WRAPPED = rf"[\(\[]?\s*{LOCATION_PATTERN}\s*[\)\]]?"
LOCATION_WRAPPED = LOCATION_PATTERN  # (simpler version here)


# ----------------------------------------------------------------------------------
# 2) MULTIPLE PATTERNS PER ORDER TYPE
# ----------------------------------------------------------------------------------

#
# 2.1 HOLD ORDERS
#
HOLD_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
(?:\s+(?P<hold_verb>{HOLD_SYNONYMS}))?
$"""

HOLD_PATTERN_B = rf"""^
(?P<hold_verb>{HOLD_SYNONYMS})
\s+
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
$"""

HOLD_PATTERNS = [
	re.compile(HOLD_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(HOLD_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.2 MOVE ORDERS
#   We'll allow something like:
#     "[unit loc] move [unit loc]" or "[unit loc] -> [unit loc]"
#     "move [unit loc] to [unit loc]"
#
MOVE_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
((?P<move_verb>{MOVE_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

MOVE_PATTERN_B = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(?:{TO_SYNONYMS})
\s+
(?P<dest>{LOCATION_WRAPPED})
$"""

MOVE_PATTERNS = [
	re.compile(MOVE_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(MOVE_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.3 SUPPORT (MOVE) ORDERS
#   e.g. "F lon supports A wal to lvp", "lon sup wal -> lvp"
#   or "support f bre moves pic"
#
SUPPORT_MOVE_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(?P<sup_verb>{SUPPORT_SYNONYMS})
\s+
((?P<tunit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<target>{LOCATION_WRAPPED})
\s+
((?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

SUPPORT_MOVE_PATTERN_B = rf"""^
((?P<sup_verb>{SUPPORT_SYNONYMS})
\s+)?
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED}):?
\s+
((?P<tunit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<target>{LOCATION_WRAPPED})
\s+
((?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

SUPPORT_MOVE_PATTERNS = [
	re.compile(SUPPORT_MOVE_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(SUPPORT_MOVE_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.4 SUPPORT (HOLD) ORDERS
#   e.g., "F lon support hold lvp", "lon s h lvp"
#
SUPPORT_HOLD_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(?P<sup_verb>{SUPPORT_SYNONYMS})
(\s*
(?:hold|h|holds|holding))?
\s+
(?P<target>{LOCATION_WRAPPED})
$"""

SUPPORT_HOLD_PATTERN_B = rf"""^
(?P<sup_verb>{SUPPORT_SYNONYMS})
(\s*
(?:hold|h|holds|holding))?
\s+
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
((?:hold|h|holds|to hold)
\s+)?
(?P<target>{LOCATION_WRAPPED})
$"""

SUPPORT_HOLD_PATTERNS = [
	re.compile(SUPPORT_HOLD_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(SUPPORT_HOLD_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.5 CONVOY ORDERS
#   e.g. "[unit loc] convoy [tunit location] to [target loc]"
#
CONVOY_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(?P<cv_verb>{CONVOY_SYNONYMS})
\s+
((?P<tunit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<target>{LOCATION_WRAPPED})
\s+
((?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

CONVOY_PATTERN_B = rf"""^
(?P<cv_verb>{CONVOY_SYNONYMS})
\s+
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED}):?
:?\s+
((?P<tunit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<target>{LOCATION_WRAPPED})
\s+
((?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

CONVOY_PATTERNS = [
	re.compile(CONVOY_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(CONVOY_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.6 BUILD ORDERS
#   e.g. "mun build", "build f mun", "army at par b"
#
BUILD_PATTERN_A = rf"""^
(?P<loc>{LOCATION_WRAPPED})
\s+
((?P<build_verb>{BUILD_SYNONYMS})
{A_AN_OPT}?
\s+)?
(?P<unit>{UNIT_PATTERN})
$
"""

BUILD_PATTERN_B = rf"""^
((?P<build_verb>{BUILD_SYNONYMS})
\s+)?
(?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+
(?P<loc>{LOCATION_WRAPPED})
$
"""

BUILD_PATTERN_C = rf"""^
(?P<loc>{LOCATION_WRAPPED})
\s+
(?P<unit>{UNIT_PATTERN})
\s+
(?P<build_verb>{BUILD_SYNONYMS})
$
"""

BUILD_PATTERN_D = rf"""^
(?P<build_verb>{BUILD_SYNONYMS})
\s+
(?P<loc>{LOCATION_WRAPPED})
{A_AN_OPT}?
\s+
(?P<unit>{UNIT_PATTERN})
$
"""


BUILD_PATTERNS = [
	re.compile(BUILD_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(BUILD_PATTERN_B, re.IGNORECASE | re.VERBOSE),
	re.compile(BUILD_PATTERN_C, re.IGNORECASE | re.VERBOSE),
	re.compile(BUILD_PATTERN_D, re.IGNORECASE | re.VERBOSE),
]

#
# 2.7 DISBAND ORDERS
#
DISBAND_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(?P<disband_verb>{DISBAND_SYNONYMS})
$
"""

DISBAND_PATTERN_B = rf"""^
((?P<disband_verb>{DISBAND_SYNONYMS})
\s+)?
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
$
"""

DISBAND_PATTERNS = [
	re.compile(DISBAND_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(DISBAND_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

#
# 2.8 RETREAT ORDERS
#   e.g. "retreat f bel to hol", "r a mun hol"
#
RETREAT_PATTERN_A = rf"""^
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
(((?P<rverb>{RETREAT_SYNONYMS})
\s+)?
(?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

RETREAT_PATTERN_B = rf"""^
(?P<rverb>{RETREAT_SYNONYMS})
\s+
((?P<unit>{UNIT_PATTERN})
{IN_AT_OPT}?
\s+)?
(?P<loc>{LOCATION_WRAPPED})
\s+
((?:{TO_SYNONYMS})
\s+)?
(?P<dest>{LOCATION_WRAPPED})
$"""

RETREAT_PATTERNS = [
	re.compile(RETREAT_PATTERN_A, re.IGNORECASE | re.VERBOSE),
	re.compile(RETREAT_PATTERN_B, re.IGNORECASE | re.VERBOSE),
]

# ----------------------------------------------------------------------------------
# 3) A DICTIONARY MAPPING ORDER TYPE -> LIST OF COMPILED PATTERNS
# ----------------------------------------------------------------------------------
ORDER_PATTERNS = {
	"hold": HOLD_PATTERNS,
	"move": MOVE_PATTERNS,
	"support_move": SUPPORT_MOVE_PATTERNS,
	"support_hold": SUPPORT_HOLD_PATTERNS,
	"convoy": CONVOY_PATTERNS,
	"build": BUILD_PATTERNS,
	"disband": DISBAND_PATTERNS,
	"retreat": RETREAT_PATTERNS,
}

ACTION_PATTERNS = {
	"hold": HOLD_PATTERNS,
	"move": MOVE_PATTERNS,
	"support_move": SUPPORT_MOVE_PATTERNS,
	"support_hold": SUPPORT_HOLD_PATTERNS,
	"convoy": CONVOY_PATTERNS,
}

GAIN_PATTERNS = {
	"build": BUILD_PATTERNS,
}

LOSE_PATTERNS = {
	"disband": DISBAND_PATTERNS,
}

RETREAT_PATTERNS = {
	"retreat": RETREAT_PATTERNS,
	"disband": DISBAND_PATTERNS,
}

# ----------------------------------------------------------------------------------
# 4) PARSE FUNCTION
# ----------------------------------------------------------------------------------

def parse_order(order_text: str, patterns) -> dict:
	"""
	Tries each order type in ORDER_PATTERNS.
	For each type, tries each pattern in that list.
	Returns a dict with:
	  {
		"type": <order_type>,
		"groups": <the named capturing groups>,
	  }
	If no pattern matches, raises a ValueError.
	"""
	text = order_text.strip()

	matches = {}
	for order_type, patterns in patterns.items():
		for pat in patterns:
			match = pat.match(text)
			if match:
				matches[order_type] = match.groupdict()
	if matches:
		return matches
	raise ValueError(f"Could not parse order: '{order_text}'.")

def run_tests(test_codes, patterns):
	tbl = []
	for order_txt in test_codes:
		try:
			ress = parse_order(order_txt, patterns)
			for typ, res in ress.items():
				tbl.append([order_txt, typ, res])
		except ValueError as e:
			tbl.append([order_txt, "FAIL", str(e)])
	print(tabulate(tbl, headers=["Order", "Match?", "Groups"]))

# ----------------------------------------------------------------------------------
# 5) DEMO / TEST
# ----------------------------------------------------------------------------------

test_action_strings = [
	# HOLD
	"F mun holds",
	"hold par",
	"par",
	"stp-nc",
	# MOVE
	"mun move par",
	"mun par",
	"nwg - stp-nc",
	"f bul-sc con",
	"navy stp -> nwg",
	# SUPPORT MOVE
	"F lon support A wal -> lvp",
	"lon support army par to spa",
	"s mun: a bel -> bur",
	"mun s bel bur",
	"mun bel bur",
	# SUPPORT HOLD
	"F lon support hold bre",
	"support f par holds mar",
	"sh mos lvp",
	"lon s h par",
	# CONVOY
	"f con convoy bul to ank",
	"convoy f lon: a bel -> pic",
	"c spa-nc: gas por",
	"mao c gas por",
	# Some random "should fail"
	"not an order at all",
	"army moving from here to there",  # doesn't fit exactly
]

run_tests(test_action_strings, ACTION_PATTERNS)

test_gain_strings = [
	# BUILD
	"mun build an a",
	"bre b a n",
	"build f par",
	"rom a",
	"f tri",
	# Some random "should fail"
	"mun",
	"not an order at all",
	"army moving from here to there",  # doesn't fit exactly
]

run_tests(test_gain_strings, GAIN_PATTERNS)

test_lose_strings = [
	# DISBAND
	"f par disband",
	"disband a mun",
	"svp",
	# Some random "should fail"
	"not an order at all",
	"army moving from here to there",  # doesn't fit exactly
]

run_tests(test_lose_strings, LOSE_PATTERNS)

test_retreat_strings = [
	# DISBAND
	"f par disband",
	"disband a mun",
	"tri",
	# RETREAT
	"f bel retreat -> hol",
	"retreat a mun to tyr",
	"tri vie",
	# Some random "should fail"
	"not an order at all",
	"army moving from here to there",  # doesn't fit exactly
]

run_tests(test_retreat_strings, RETREAT_PATTERNS)

Order                           Match?        Groups
------------------------------  ------------  ----------------------------------------------------------------------------------------------------
F mun holds                     hold          {'unit': 'F', 'loc': 'mun', 'hold_verb': 'holds'}
hold par                        hold          {'hold_verb': 'hold', 'unit': None, 'loc': 'par'}
par                             hold          {'unit': None, 'loc': 'par', 'hold_verb': None}
stp-nc                          hold          {'unit': None, 'loc': 'stp-nc', 'hold_verb': None}
mun move par                    move          {'unit': None, 'loc': 'mun', 'move_verb': 'move', 'dest': 'par'}
mun par                         move          {'unit': None, 'loc': 'mun', 'move_verb': None, 'dest': 'par'}
nwg - stp-nc                    move          {'unit': None, 'loc': 'nwg', 'dest': 'stp-nc'}
f bul-sc con                    move          {'unit': 'f', 'loc': 'bul-sc', 'move_verb': None, 'dest': 

In [112]:
type(RETREAT_PATTERNS['retreat'][0])

re.Pattern